# Reinforcement Learning
## Some pre-requisites if running on Google Collab
If not running on Google collab do not run these next two cells!

In [ ]:
# Install the only dependency not available from collab directly
!pip install chess

# Get imported files from repo
!git clone -b rl-setup https://github.com/owenjaques/chessbot.git
!mv chessbot chessbot-repo
!mv chessbot-repo/src/chessbot .
!rm chessbot-repo -r

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')
weights_directory = '/content/gdrive/MyDrive/chessbot_weights/'
print(f'Saving weights to {weights_directory}')

## If not running on Google Collab
Set the weights directory variable to wherever you would like data saved.

In [ ]:
weights_directory = 'your directory here'

## Our model
In this section initialize your model. A fairly standard neural network is currently set up there.

In [ ]:
from tensorflow import keras

model = keras.Sequential([
	keras.layers.Dense(64, activation='relu'),
	keras.layers.Dense(64, activation='relu'),
	keras.layers.Dense(1)
])

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.01),
    loss='mse'
)

## Setting the exploration rate
The exploration rate is defined as the chance of randomly making a move instead of relying on the prediction of the model.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

num_games = 10000
exploration_rates = np.linspace(1, 0, num_games)**2

fig, ax = plt.subplots(figsize=(12, 5))

ax.set_xlabel('Number of games played')
ax.set_ylabel('Exploration rate')
ax.plot(np.linspace(1, num_games, num_games), exploration_rates)
ax.set_title('Exploration rate as the number of games played increases')

plt.show()

## Training the model
The model will be trained by playing against itself.

In [ ]:
import chess
import numpy as np
import time
from IPython.display import clear_output
from chessbot.chessbot import ChessBot

def play_game_and_learn(model, exploration_rate=0.0, should_visualise=False):
	white = ChessBot(model, chess.WHITE, exploration_rate)
	black = ChessBot(model, chess.BLACK, exploration_rate)

	board = chess.Board()

	if should_visualise:
		display(board)

	while not board.is_game_over(claim_draw=True):
		board.push(black.move(board) if board.turn == chess.BLACK else white.move(board))

		if should_visualise:
			clear_output(wait=True)
			display(board)
			time.sleep(0.5)

	result = board.outcome(claim_draw=True).result()

	# If the game was not a draw train the model
	if result == '1/2-1/2':
		return result, None, None
	
	X = np.empty((len(white.moves_made) + len(black.moves_made), len(white.moves_made[0])))
	y = np.empty_like(X)

	# Blend the moves into a single array with alternating elements
	X[::2] = white.moves_made
	X[1::2] = black.moves_made

	# Set the label for the last move to 1, representing a winning move, then
	# discount the rest as they led to a win but should not be rewarded as heavily
	discount_factor = 0.95
	y_reversed_indices = np.linspace(len(y) - 1, 0, num=len(y))
	y = 1 * discount_factor**y_reversed_indices

	# If black won flip the labels since our model evaluates white's position
	if result == '0-1':
		y *= -1

	# Scale the labels to be between 0 and 1 instead of -1 and 1
	y = (y + 1) / 2
		
	model.fit(X, y)

	return result, X, y

In [ ]:
# Initialize objects to track game results (only run this cell when you want to reset results)
results = []
white_wins = 0
black_wins = 0
draws = 0

X_all = None
y_all = None

In [ ]:
from IPython.display import clear_output

for i in range(5116, num_games):
	clear_output(wait=True)
	if i > 0:
		print(f"The last game's result: {results[-1]}")
	print(f'White wins: {white_wins}, Black wins: {black_wins}, Draws: {draws}')
	print(f'Game {i + 1}/{num_games} (exploration rate: {exploration_rates[i]:.2f})')

	result, X, y = play_game_and_learn(model, exploration_rate=exploration_rates[i])
	results.append(result)

	if X is not None and y is not None:
		if X_all is None:
			X_all = X
			y_all = y
		else:
			X_all = np.concatenate((X_all, X))
			y_all = np.concatenate((y_all, y))

	if result == '1-0':
		white_wins += 1
	if result == '0-1':
		black_wins += 1
	if result == '1/2-1/2':
		draws += 1

	# Save the weights and training data every 100 games
	if (i + 1) % 1000 == 0:
		model.save(f'{weights_directory}{i + 1}_games_model')
		np.savez_compressed(f'{weights_directory}games_data.npz', X=X_all, y=y_all)

## Why not watch a game after all that training?

In [ ]:
result = play_game_and_learn(model, should_visualise=True)
print(f'Game result: {result}')